In [14]:
import numpy as np
import pandas as pd
import os
import tarfile
import io
import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.preprocessing import OrdinalEncoder

In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [16]:
i=0
for column in train:
  print("attribute ", i, "named ", column, "missing count: ", train[column].isna().sum())
  i += 1

attribute  0 named  Id missing count:  0
attribute  1 named  MSSubClass missing count:  0
attribute  2 named  MSZoning missing count:  0
attribute  3 named  LotFrontage missing count:  259
attribute  4 named  LotArea missing count:  0
attribute  5 named  Street missing count:  0
attribute  6 named  Alley missing count:  1369
attribute  7 named  LotShape missing count:  0
attribute  8 named  LandContour missing count:  0
attribute  9 named  Utilities missing count:  0
attribute  10 named  LotConfig missing count:  0
attribute  11 named  LandSlope missing count:  0
attribute  12 named  Neighborhood missing count:  0
attribute  13 named  Condition1 missing count:  0
attribute  14 named  Condition2 missing count:  0
attribute  15 named  BldgType missing count:  0
attribute  16 named  HouseStyle missing count:  0
attribute  17 named  OverallQual missing count:  0
attribute  18 named  OverallCond missing count:  0
attribute  19 named  YearBuilt missing count:  0
attribute  20 named  YearRemo

In [17]:
train = train.drop(columns=['LotFrontage','Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'])
test = test.drop(columns=['LotFrontage','Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'])

In [18]:
train = train.dropna(subset=['MasVnrType', 'Electrical', 'BsmtFinType2', 'BsmtExposure', 'GarageCond'])
i=0
for column in train:
  print("attribute ", i, "named ", column, "missing count: ", train[column].isna().sum())
  i += 1

attribute  0 named  Id missing count:  0
attribute  1 named  MSSubClass missing count:  0
attribute  2 named  MSZoning missing count:  0
attribute  3 named  LotArea missing count:  0
attribute  4 named  Street missing count:  0
attribute  5 named  LotShape missing count:  0
attribute  6 named  LandContour missing count:  0
attribute  7 named  Utilities missing count:  0
attribute  8 named  LotConfig missing count:  0
attribute  9 named  LandSlope missing count:  0
attribute  10 named  Neighborhood missing count:  0
attribute  11 named  Condition1 missing count:  0
attribute  12 named  Condition2 missing count:  0
attribute  13 named  BldgType missing count:  0
attribute  14 named  HouseStyle missing count:  0
attribute  15 named  OverallQual missing count:  0
attribute  16 named  OverallCond missing count:  0
attribute  17 named  YearBuilt missing count:  0
attribute  18 named  YearRemodAdd missing count:  0
attribute  19 named  RoofStyle missing count:  0
attribute  20 named  RoofMatl

In [19]:
print(len(train['Id']))

1338


In [20]:
train = train.drop(columns=['Id'])
test = test.drop(columns=['Id'])

In [21]:
train['MSSubClass'].head()

0    60
1    20
2    60
3    70
4    60
Name: MSSubClass, dtype: int64

In [22]:
type(train['LotShape'])

pandas.core.series.Series

In [23]:
train['LotShape'].replace(to_replace='IR3', value=0, inplace=True)

In [24]:
train['LotShape'].replace(to_replace='IR2', value=1, inplace=True)
train['LotShape'].replace(to_replace='IR1', value=2, inplace=True)
train['LotShape'].replace(to_replace='Reg', value=3, inplace=True)

In [25]:
test['LotShape'].replace(to_replace='IR3', value=0, inplace=True)
test['LotShape'].replace(to_replace='IR2', value=1, inplace=True)
test['LotShape'].replace(to_replace='IR1', value=2, inplace=True)
test['LotShape'].replace(to_replace='Reg', value=3, inplace=True)

In [26]:
train['LotShape'].head()

0    3
1    3
2    2
3    2
4    2
Name: LotShape, dtype: int64

In [27]:
train['Street'].replace(to_replace='Grvl', value=0, inplace=True)
train['Street'].replace(to_replace='Pave', value=1, inplace=True)

In [28]:
test['Street'].replace(to_replace='Grvl', value=0, inplace=True)
test['Street'].replace(to_replace='Pave', value=1, inplace=True)

In [29]:
train['LandContour'].replace(to_replace='Low', value=0, inplace=True)
train['LandContour'].replace(to_replace='HLS', value=1, inplace=True)
train['LandContour'].replace(to_replace='Bnk', value=2, inplace=True)
train['LandContour'].replace(to_replace='Lvl', value=3, inplace=True)

In [30]:
train['Utilities'].replace(to_replace='ELO', value=0, inplace=True)
train['Utilities'].replace(to_replace='NoSewa', value=1, inplace=True)
train['Utilities'].replace(to_replace='NoSewr', value=2, inplace=True)
train['Utilities'].replace(to_replace='AllPub', value=3, inplace=True)


In [31]:
test['LandContour'].replace(to_replace='Low', value=0, inplace=True)
test['LandContour'].replace(to_replace='HLS', value=1, inplace=True)
test['LandContour'].replace(to_replace='Bnk', value=2, inplace=True)
test['LandContour'].replace(to_replace='Lvl', value=3, inplace=True)
test['Utilities'].replace(to_replace='ELO', value=0, inplace=True)
test['Utilities'].replace(to_replace='NoSewa', value=1, inplace=True)
test['Utilities'].replace(to_replace='NoSewr', value=2, inplace=True)
test['Utilities'].replace(to_replace='AllPub', value=3, inplace=True)


In [32]:
train.drop(columns=['LotConfig'])

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LandSlope,Neighborhood,Condition1,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,8450,1,3,3,3,Gtl,CollgCr,Norm,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,9600,1,3,3,3,Gtl,Veenker,Feedr,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,11250,1,2,3,3,Gtl,CollgCr,Norm,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,9550,1,2,3,3,Gtl,Crawfor,Norm,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,14260,1,2,3,3,Gtl,NoRidge,Norm,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,7917,1,3,3,3,Gtl,Gilbert,Norm,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,13175,1,3,3,3,Gtl,NWAmes,Norm,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,9042,1,3,3,3,Gtl,Crawfor,Norm,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,9717,1,3,3,3,Gtl,NAmes,Norm,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [33]:
test.drop(columns=['LotConfig'])

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LandSlope,Neighborhood,Condition1,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,11622,1,3,3,3.0,Gtl,NAmes,Feedr,...,0,0,0,120,0,0,6,2010,WD,Normal
1,20,RL,14267,1,2,3,3.0,Gtl,NAmes,Norm,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,60,RL,13830,1,2,3,3.0,Gtl,Gilbert,Norm,...,34,0,0,0,0,0,3,2010,WD,Normal
3,60,RL,9978,1,2,3,3.0,Gtl,Gilbert,Norm,...,36,0,0,0,0,0,6,2010,WD,Normal
4,120,RL,5005,1,2,1,3.0,Gtl,StoneBr,Norm,...,82,0,0,144,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,1936,1,3,3,3.0,Gtl,MeadowV,Norm,...,0,0,0,0,0,0,6,2006,WD,Normal
1455,160,RM,1894,1,3,3,3.0,Gtl,MeadowV,Norm,...,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,20,RL,20000,1,3,3,3.0,Gtl,Mitchel,Norm,...,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,85,RL,10441,1,3,3,3.0,Gtl,Mitchel,Norm,...,32,0,0,0,0,700,7,2006,WD,Normal


In [34]:
train['LandSlope'].replace(to_replace='Sev', value=0, inplace=True)


In [35]:
train['LandSlope'].replace(to_replace='Mod', value=1, inplace=True)
train['LandSlope'].replace(to_replace='Gtl', value=2, inplace=True)

test['LandSlope'].replace(to_replace='Sev', value=0, inplace=True)
test['LandSlope'].replace(to_replace='Mod', value=1, inplace=True)
test['LandSlope'].replace(to_replace='Gtl', value=2, inplace=True)

In [36]:
train['ExterQual'].replace(to_replace='Po', value=0, inplace=True)
train['ExterQual'].replace(to_replace='Fa', value=1, inplace=True)
train['ExterQual'].replace(to_replace='TA', value=2, inplace=True)
train['ExterQual'].replace(to_replace='Gd', value=3, inplace=True)
train['ExterQual'].replace(to_replace='Ex', value=4, inplace=True)

test['ExterQual'].replace(to_replace='Po', value=0, inplace=True)
test['ExterQual'].replace(to_replace='Fa', value=1, inplace=True)
test['ExterQual'].replace(to_replace='TA', value=2, inplace=True)
test['ExterQual'].replace(to_replace='Gd', value=3, inplace=True)
test['ExterQual'].replace(to_replace='Ex', value=4, inplace=True)

In [37]:
train['ExterCond'].replace(to_replace='Po', value=0, inplace=True)
train['ExterCond'].replace(to_replace='Fa', value=1, inplace=True)
train['ExterCond'].replace(to_replace='TA', value=2, inplace=True)
train['ExterCond'].replace(to_replace='Gd', value=3, inplace=True)
train['ExterCond'].replace(to_replace='Ex', value=4, inplace=True)

test['ExterCond'].replace(to_replace='Po', value=0, inplace=True)
test['ExterCond'].replace(to_replace='Fa', value=1, inplace=True)
test['ExterCond'].replace(to_replace='TA', value=2, inplace=True)
test['ExterCond'].replace(to_replace='Gd', value=3, inplace=True)
test['ExterCond'].replace(to_replace='Ex', value=4, inplace=True)

In [38]:
train['BsmtQual'].replace(to_replace='NA', value=-1, inplace=True)
train['BsmtQual'].replace(to_replace='Po', value=0, inplace=True)
train['BsmtQual'].replace(to_replace='Fa', value=1, inplace=True)
train['BsmtQual'].replace(to_replace='TA', value=2, inplace=True)
train['BsmtQual'].replace(to_replace='Gd', value=3, inplace=True)
train['BsmtQual'].replace(to_replace='Ex', value=4, inplace=True)

test['BsmtQual'].replace(to_replace='NA', value=-1, inplace=True)
test['BsmtQual'].replace(to_replace='Po', value=0, inplace=True)
test['BsmtQual'].replace(to_replace='Fa', value=1, inplace=True)
test['BsmtQual'].replace(to_replace='TA', value=2, inplace=True)
test['BsmtQual'].replace(to_replace='Gd', value=3, inplace=True)
test['BsmtQual'].replace(to_replace='Ex', value=4, inplace=True)

In [39]:
train['BsmtCond'].replace(to_replace='NA', value=-1, inplace=True)
train['BsmtCond'].replace(to_replace='Po', value=0, inplace=True)
train['BsmtCond'].replace(to_replace='Fa', value=1, inplace=True)
train['BsmtCond'].replace(to_replace='TA', value=2, inplace=True)
train['BsmtCond'].replace(to_replace='Gd', value=3, inplace=True)
train['BsmtCond'].replace(to_replace='Ex', value=4, inplace=True)

test['BsmtCond'].replace(to_replace='NA', value=-1, inplace=True)
test['BsmtCond'].replace(to_replace='Po', value=0, inplace=True)
test['BsmtCond'].replace(to_replace='Fa', value=1, inplace=True)
test['BsmtCond'].replace(to_replace='TA', value=2, inplace=True)
test['BsmtCond'].replace(to_replace='Gd', value=3, inplace=True)
test['BsmtCond'].replace(to_replace='Ex', value=4, inplace=True)

In [40]:
train['BsmtExposure'].replace(to_replace='NA', value=0, inplace=True)
train['BsmtExposure'].replace(to_replace='No', value=1, inplace=True)
train['BsmtExposure'].replace(to_replace='Mn', value=2, inplace=True)
train['BsmtExposure'].replace(to_replace='Av', value=3, inplace=True)
train['BsmtExposure'].replace(to_replace='Gd', value=4, inplace=True)

test['BsmtExposure'].replace(to_replace='NA', value=0, inplace=True)
test['BsmtExposure'].replace(to_replace='No', value=1, inplace=True)
test['BsmtExposure'].replace(to_replace='Mn', value=2, inplace=True)
test['BsmtExposure'].replace(to_replace='Av', value=3, inplace=True)
test['BsmtExposure'].replace(to_replace='Gd', value=4, inplace=True)

In [41]:
train['BsmtFinType1'].replace(to_replace='NA', value=-1, inplace=True)
train['BsmtFinType1'].replace(to_replace='Unf', value=0, inplace=True)
train['BsmtFinType1'].replace(to_replace='Fa', value=1, inplace=True)
train['BsmtFinType1'].replace(to_replace='LwQ', value=2, inplace=True)
train['BsmtFinType1'].replace(to_replace='Rec', value=3, inplace=True)
train['BsmtFinType1'].replace(to_replace='BLQ', value=4, inplace=True)
train['BsmtFinType1'].replace(to_replace='ALQ', value=5, inplace=True)
train['BsmtFinType1'].replace(to_replace='GLQ', value=6, inplace=True)

test['BsmtFinType1'].replace(to_replace='NA', value=-1, inplace=True)
test['BsmtFinType1'].replace(to_replace='Unf', value=0, inplace=True)
test['BsmtFinType1'].replace(to_replace='Fa', value=1, inplace=True)
test['BsmtFinType1'].replace(to_replace='LwQ', value=2, inplace=True)
test['BsmtFinType1'].replace(to_replace='Rec', value=3, inplace=True)
test['BsmtFinType1'].replace(to_replace='BLQ', value=4, inplace=True)
test['BsmtFinType1'].replace(to_replace='ALQ', value=5, inplace=True)
test['BsmtFinType1'].replace(to_replace='GLQ', value=6, inplace=True)

In [42]:
train['BsmtFinType2'].replace(to_replace='NA', value=-1, inplace=True)
train['BsmtFinType2'].replace(to_replace='Unf', value=0, inplace=True)
train['BsmtFinType2'].replace(to_replace='Fa', value=1, inplace=True)
train['BsmtFinType2'].replace(to_replace='LwQ', value=2, inplace=True)
train['BsmtFinType2'].replace(to_replace='Rec', value=3, inplace=True)
train['BsmtFinType2'].replace(to_replace='BLQ', value=4, inplace=True)
train['BsmtFinType2'].replace(to_replace='ALQ', value=5, inplace=True)
train['BsmtFinType2'].replace(to_replace='GLQ', value=6, inplace=True)

test['BsmtFinType2'].replace(to_replace='NA', value=-1, inplace=True)
test['BsmtFinType2'].replace(to_replace='Unf', value=0, inplace=True)
test['BsmtFinType2'].replace(to_replace='Fa', value=1, inplace=True)
test['BsmtFinType2'].replace(to_replace='LwQ', value=2, inplace=True)
test['BsmtFinType2'].replace(to_replace='Rec', value=3, inplace=True)
test['BsmtFinType2'].replace(to_replace='BLQ', value=4, inplace=True)
test['BsmtFinType2'].replace(to_replace='ALQ', value=5, inplace=True)
test['BsmtFinType2'].replace(to_replace='GLQ', value=6, inplace=True)

In [43]:
train['HeatingQC'].replace(to_replace='Po', value=0, inplace=True)
train['HeatingQC'].replace(to_replace='Fa', value=1, inplace=True)
train['HeatingQC'].replace(to_replace='TA', value=2, inplace=True)
train['HeatingQC'].replace(to_replace='Gd', value=3, inplace=True)
train['HeatingQC'].replace(to_replace='Ex', value=4, inplace=True)

test['HeatingQC'].replace(to_replace='Po', value=0, inplace=True)
test['HeatingQC'].replace(to_replace='Fa', value=1, inplace=True)
test['HeatingQC'].replace(to_replace='TA', value=2, inplace=True)
test['HeatingQC'].replace(to_replace='Gd', value=3, inplace=True)
test['HeatingQC'].replace(to_replace='Ex', value=4, inplace=True)

In [44]:
train['CentralAir'].replace(to_replace='N', value=0, inplace=True)
train['CentralAir'].replace(to_replace='Y', value=1, inplace=True)

test['CentralAir'].replace(to_replace='N', value=0, inplace=True)
test['CentralAir'].replace(to_replace='Y', value=1, inplace=True)

In [45]:
train['KitchenQual'].replace(to_replace='Po', value=0, inplace=True)
train['KitchenQual'].replace(to_replace='Fa', value=1, inplace=True)
train['KitchenQual'].replace(to_replace='TA', value=2, inplace=True)
train['KitchenQual'].replace(to_replace='Gd', value=3, inplace=True)
train['KitchenQual'].replace(to_replace='Ex', value=4, inplace=True)

test['KitchenQual'].replace(to_replace='Po', value=0, inplace=True)
test['KitchenQual'].replace(to_replace='Fa', value=1, inplace=True)
test['KitchenQual'].replace(to_replace='TA', value=2, inplace=True)
test['KitchenQual'].replace(to_replace='Gd', value=3, inplace=True)
test['KitchenQual'].replace(to_replace='Ex', value=4, inplace=True)

In [46]:
train['Functional'].replace(to_replace='Sal', value=0, inplace=True)
train['Functional'].replace(to_replace='Sev', value=1, inplace=True)
train['Functional'].replace(to_replace='Maj2', value=2, inplace=True)
train['Functional'].replace(to_replace='Maj1', value=3, inplace=True)
train['Functional'].replace(to_replace='Mod', value=4, inplace=True)
train['Functional'].replace(to_replace='Min2', value=5, inplace=True)
train['Functional'].replace(to_replace='Min1', value=6, inplace=True)
train['Functional'].replace(to_replace='Typ', value=7, inplace=True)


test['Functional'].replace(to_replace='Sal', value=0, inplace=True)
test['Functional'].replace(to_replace='Sev', value=1, inplace=True)
test['Functional'].replace(to_replace='Maj2', value=2, inplace=True)
test['Functional'].replace(to_replace='Maj1', value=3, inplace=True)
test['Functional'].replace(to_replace='Mod', value=4, inplace=True)
test['Functional'].replace(to_replace='Min2', value=5, inplace=True)
test['Functional'].replace(to_replace='Min1', value=6, inplace=True)
test['Functional'].replace(to_replace='Typ', value=7, inplace=True)

In [47]:
train['GarageFinish'].replace(to_replace='NA', value=-1, inplace=True)
train['GarageFinish'].replace(to_replace='Unf', value=0, inplace=True)
train['GarageFinish'].replace(to_replace='RFn', value=1, inplace=True)
train['GarageFinish'].replace(to_replace='Fin', value=2, inplace=True)

test['GarageFinish'].replace(to_replace='NA', value=-1, inplace=True)
test['GarageFinish'].replace(to_replace='Unf', value=0, inplace=True)
test['GarageFinish'].replace(to_replace='RFn', value=1, inplace=True)
test['GarageFinish'].replace(to_replace='Fin', value=2, inplace=True)

In [48]:
train['GarageQual'].replace(to_replace='NA', value=-1, inplace=True)
train['GarageQual'].replace(to_replace='Po', value=0, inplace=True)
train['GarageQual'].replace(to_replace='Fa', value=1, inplace=True)
train['GarageQual'].replace(to_replace='TA', value=2, inplace=True)
train['GarageQual'].replace(to_replace='Gd', value=3, inplace=True)
train['GarageQual'].replace(to_replace='Ex', value=4, inplace=True)

test['GarageQual'].replace(to_replace='NA', value=-1, inplace=True)
test['GarageQual'].replace(to_replace='Po', value=0, inplace=True)
test['GarageQual'].replace(to_replace='Fa', value=1, inplace=True)
test['GarageQual'].replace(to_replace='TA', value=2, inplace=True)
test['GarageQual'].replace(to_replace='Gd', value=3, inplace=True)
test['GarageQual'].replace(to_replace='Ex', value=4, inplace=True)

In [49]:
train['GarageCond'].replace(to_replace='NA', value=-1, inplace=True)
train['GarageCond'].replace(to_replace='Po', value=0, inplace=True)
train['GarageCond'].replace(to_replace='Fa', value=1, inplace=True)
train['GarageCond'].replace(to_replace='TA', value=2, inplace=True)
train['GarageCond'].replace(to_replace='Gd', value=3, inplace=True)
train['GarageCond'].replace(to_replace='Ex', value=4, inplace=True)

test['GarageCond'].replace(to_replace='NA', value=-1, inplace=True)
test['GarageCond'].replace(to_replace='Po', value=0, inplace=True)
test['GarageCond'].replace(to_replace='Fa', value=1, inplace=True)
test['GarageCond'].replace(to_replace='TA', value=2, inplace=True)
test['GarageCond'].replace(to_replace='Gd', value=3, inplace=True)
test['GarageCond'].replace(to_replace='Ex', value=4, inplace=True)

In [50]:
train['PavedDrive'].replace(to_replace='N', value=0, inplace=True)
train['PavedDrive'].replace(to_replace='P', value=1, inplace=True)
train['PavedDrive'].replace(to_replace='Y', value=2, inplace=True)

test['PavedDrive'].replace(to_replace='N', value=0, inplace=True)
test['PavedDrive'].replace(to_replace='P', value=1, inplace=True)
test['PavedDrive'].replace(to_replace='Y', value=2, inplace=True)

Τωρα ξεκιναμε με τα One Hot:

'MSSubClass'
'MSZoning'
'Neighborhood'
'Condition1'
'Condition2'
'BldgType'
'RoofStyle'
'RoofMatl'
'Exterior1st'
'Exterior2nd'
'MasVnrType'
'Foundation'
'Heating'
'Electrical'
'GarageType'
'SaleType'
'SaleCondition'